# Testing the model

### Load the model

In [2]:
from tensorflow.keras.models import load_model
model_loaded = load_model("RSP_weights_C.h5")
model_loaded.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 208, 176, 5)       185       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 104, 88, 5)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 104, 88, 15)       315       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 52, 44, 15)        0         
_________________________________________________________________
flatten (Flatten)            (None, 34320)             0         
_________________________________________________________________
dense (Dense)                (None, 500)               17160500  
_________________________________________________________________
batch_normalization (BatchNo (None, 500)               2

### Import the needed packages

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [4]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array 

### Import test images and assign category

In [7]:
base_path = "../data/alzheimer_mri/test"
classes = ["VeryMildDemented", "NonDemented", "ModerateDemented", "MildDemented"]

X_test =[]
y_true= []
images=[]

for i, target in enumerate(classes):
    files = os.listdir(f"{base_path}/{target}")
    for file in files:
        image_path=f"{base_path}/{target}/{file}"
        #Load image
        image = load_img(image_path, color_mode="grayscale")
        #Convert image to array 
        image_array = img_to_array(image)
        #Append to X
        X_test.append(image_array)
        # append the numeric target to y
        y_true.append(i)

#Concert to array 
X_test_array = np.array(X_test)
y_ture_array = np.array(y_true)

In [8]:
X_test_array.shape

(1279, 208, 176, 1)

### Make prediction on test images based on previously trained CNN model

In [9]:
y_pred = model_loaded.predict(X_test_array)

In [11]:
#Assign true labels

y_true_labels=[]
for i in y_true:
    if i ==0:
        y_true_label = "VeryMildDemented"
        y_true_labels.append(y_true_label) 
    elif i ==1:    
        y_true_label ="NonDemented"
        y_true_labels.append(y_true_label) 
    elif i ==2:    
        y_true_label ="ModerateDemented"
        y_true_labels.append(y_true_label)
    elif i ==3:    
        y_true_label ="MildDemented"
        y_true_labels.append(y_true_label)
    else:
        y_true_label = "prediction (y) not properly assigned"
        y_true_labels.append(y_true_label)

In [12]:
#Assign predicted labels and the probability

y_pred_labels=[]
#suggested_category=[]
likelyhood = [] 
for i in y_pred:
    perc = round((np.max(i)*100), 2)
    likelyhood.append(perc)
    if np.argmax(i) ==0:
        ypred_label = "VeryMildDemented" 
        y_pred_labels.append(ypred_label)
    elif np.argmax(i) ==1:
        ypred_label = "NonDemented" 
        y_pred_labels.append(ypred_label)
    elif np.argmax(i) ==2:
        ypred_label = "ModerateDemented"
        y_pred_labels.append(ypred_label)
    elif np.argmax(i) ==3:
        ypred_label = "MildDemented"
        y_pred_labels.append(ypred_label)     
    else:
        ypred_label = "prediction (y) not properly assigned"
y_pred_df = pd.DataFrame(list(zip(y_pred_labels, likelyhood, y_true_labels)), columns=["prediction", "likelyhood", "true label"])

### Explore the prediction in a dataframe

In [13]:
y_pred_df

,prediction,likelyhood,true label
0,VeryMildDemented,94.02,VeryMildDemented
1,MildDemented,53.60,VeryMildDemented
2,VeryMildDemented,98.98,VeryMildDemented
3,VeryMildDemented,99.96,VeryMildDemented
4,VeryMildDemented,98.77,VeryMildDemented
...,...,...,...
1274,MildDemented,88.42,MildDemented
1275,MildDemented,73.18,MildDemented
1276,MildDemented,51.35,MildDemented
1277,VeryMildDemented,91.52,MildDemented


In [14]:
#define conditions
conditions = [
    y_pred_df['prediction'] == y_pred_df['true label'],
    y_pred_df['prediction'] != y_pred_df['true label']
    ]

#define choices
category = [1, 0] #1 - correct; 0 -false 

#create new column in DataFrame that displays results of comparisons
y_pred_df["correct"]=np.select(conditions, category, default=0) 

In [15]:
#Check if the predictions were correct (1/0)

y_pred_df

,prediction,likelyhood,true label,correct
0,VeryMildDemented,94.02,VeryMildDemented,1
1,MildDemented,53.60,VeryMildDemented,0
2,VeryMildDemented,98.98,VeryMildDemented,1
3,VeryMildDemented,99.96,VeryMildDemented,1
4,VeryMildDemented,98.77,VeryMildDemented,1
...,...,...,...,...
1274,MildDemented,88.42,MildDemented,1
1275,MildDemented,73.18,MildDemented,1
1276,MildDemented,51.35,MildDemented,1
1277,VeryMildDemented,91.52,MildDemented,0


In [17]:
prediction_counts = y_pred_df['correct'].value_counts()
prediction_counts

1    730
0    549
Name: correct, dtype: int64

In [20]:
corr_perc=round((prediction_counts[1]/(prediction_counts[1]+prediction_counts[0]))*100, 2)
print(f"The model predicted {corr_perc}% of the unseen test images class")

The model predicted 57.08% of the unseen test images class


In [23]:
#define conditions
conditions_2 = [
    ((y_pred_df['prediction'].isin(["VeryMildDemented", "ModerateDemented", "MildDemented"])) &
    (y_pred_df['true label'].isin(["VeryMildDemented", "ModerateDemented", "MildDemented"]))),
    ((y_pred_df['prediction'] == "NonDemented" ) &
    (y_pred_df['true label'] == "NonDemented")),
    ]

#define choices
category_2 = [1, 1] #1 - correct; 1 - correct

#create new column in DataFrame that displays results of comparisons
y_pred_df["correct_demented"]=np.select(conditions_2, category_2, default=0) 

In [24]:
y_pred_df

,prediction,likelyhood,true label,correct,correct_demented
0,VeryMildDemented,94.02,VeryMildDemented,1,1
1,MildDemented,53.60,VeryMildDemented,0,1
2,VeryMildDemented,98.98,VeryMildDemented,1,1
3,VeryMildDemented,99.96,VeryMildDemented,1,1
4,VeryMildDemented,98.77,VeryMildDemented,1,1
...,...,...,...,...,...
1274,MildDemented,88.42,MildDemented,1,1
1275,MildDemented,73.18,MildDemented,1,1
1276,MildDemented,51.35,MildDemented,1,1
1277,VeryMildDemented,91.52,MildDemented,0,1


In [25]:
y_predictions_model_C = y_pred_df

In [29]:
y_predictions_model_C.rename(columns = {'prediction':'Predicted label', 
'likelyhood': 'Likelihood, %', 
'true label':'True label', 
'correct':'If the subcategory is assign correctly 1/0', 
'correct_demented': 'If Dementia is predicted correctly 1/0'}, inplace=True)

In [30]:
y_predictions_model_C

,Predicted label,"Likelihood, %",True label,If the subcategory is assign correctly 1/0,If Dementia is predicted correctly 1/0
0,VeryMildDemented,94.02,VeryMildDemented,1,1
1,MildDemented,53.60,VeryMildDemented,0,1
2,VeryMildDemented,98.98,VeryMildDemented,1,1
3,VeryMildDemented,99.96,VeryMildDemented,1,1
4,VeryMildDemented,98.77,VeryMildDemented,1,1
...,...,...,...,...,...
1274,MildDemented,88.42,MildDemented,1,1
1275,MildDemented,73.18,MildDemented,1,1
1276,MildDemented,51.35,MildDemented,1,1
1277,VeryMildDemented,91.52,MildDemented,0,1


In [ ]:
#Save the dataframe
y_predictions_model_C.to_csv('../data/Testing_CNN_model.csv', index=False)

In [32]:
prediction_demented_counts = y_pred_df['If Dementia is predicted correctly 1/0'].value_counts()
prediction_demented_counts

1    876
0    403
Name: If Dementia is predicted correctly 1/0, dtype: int64

In [33]:
corr_demented_perc=round((prediction_demented_counts[1]/(prediction_demented_counts[1]+prediction_demented_counts[0]))*100, 2)
print(f"The model predicted {corr_demented_perc}% of the unseen test images correctly as demented (nondemented)")

The model predicted 68.49% of the unseen test images correctly as demented (nondemented)
